## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

(50000, 32, 32, 3)


In [3]:
# Normalization
def norm(x):
    x = x/255.0
    return x

x_train = norm(x_train)
x_test = norm(x_test)

In [5]:
# onehot encodeing
n_class = 10
y_train = to_categorical(y_train, n_class)
y_test = to_categorical(y_test, n_class)

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, shuffle=True)
print(f'x_train.shape: {x_train.shape}   y_train.shape: {y_train.shape}')
print(f'x_val.shape  : {x_val.shape}   y_val.shape  : {y_val.shape}')

x_train.shape: (40000, 32, 32, 3)   y_train.shape: (40000, 10)
x_val.shape  : (10000, 32, 32, 3)   y_val.shape  : (10000, 10)


In [16]:

classifier=Sequential()
input_shape = (32, 32, 3)
#卷積組合
classifier.add(Convolution2D(filters=32, 
                             kernel_size=(3, 3), 
                             padding='same', 
                             input_shape=input_shape))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.95, epsilon=0.001,))

classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same'))
classifier.add(BatchNormalization(momentum=0.95, epsilon=0.001,))
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32, 
                             kernel_size=(3, 3), 
                             padding='same'))
classifier.add(BatchNormalization(momentum=0.95, epsilon=0.001,))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=128,activation='relu'))

#輸出
classifier.add(Dense(units=n_class,activation='softmax'))

classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)       

In [17]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=40,epochs=40)

Epoch 1/40
1000/1000 [==============================] - 185s 185ms/step - loss: 1.5722 - accuracy: 0.4447
Epoch 2/40
1000/1000 [==============================] - 238s 238ms/step - loss: 1.1437 - accuracy: 0.6003
Epoch 3/40
1000/1000 [==============================] - 205s 205ms/step - loss: 0.9972 - accuracy: 0.6514
Epoch 4/40
1000/1000 [==============================] - 218s 218ms/step - loss: 0.8927 - accuracy: 0.6884
Epoch 5/40
1000/1000 [==============================] - 227s 227ms/step - loss: 0.8045 - accuracy: 0.7201
Epoch 6/40
1000/1000 [==============================] - 225s 225ms/step - loss: 0.7233 - accuracy: 0.7465
Epoch 7/40
1000/1000 [==============================] - 211s 211ms/step - loss: 0.6458 - accuracy: 0.7745
Epoch 8/40
1000/1000 [==============================] - 232s 232ms/step - loss: 0.5649 - accuracy: 0.8007
Epoch 9/40
1000/1000 [==============================] - 225s 225ms/step - loss: 0.4864 - accuracy: 0.8318
Epoch 10/40
1000/1000 [=======================

In [41]:
y_pred = classifier.predict(x_val)
y_pred

array([[1.6151066e-09, 1.2966266e-09, 1.9070538e-13, ..., 2.1066679e-10,
        8.1977610e-13, 9.9928802e-01],
       [2.0094356e-21, 1.3440259e-15, 8.5677544e-24, ..., 5.3207163e-30,
        1.0000000e+00, 6.8329423e-14],
       [3.0047799e-05, 2.5707032e-16, 1.7698890e-09, ..., 4.6172151e-18,
        9.9996996e-01, 1.7684090e-15],
       ...,
       [2.4426094e-14, 4.1078292e-16, 1.3528060e-38, ..., 0.0000000e+00,
        1.0000000e+00, 2.5518735e-24],
       [3.5455052e-17, 1.5624350e-17, 3.5119663e-10, ..., 1.3961680e-09,
        7.2356752e-20, 3.2852360e-20],
       [1.4343563e-17, 3.7459074e-13, 1.2625048e-07, ..., 9.6043351e-14,
        1.1896689e-15, 4.1809761e-16]], dtype=float32)

In [42]:
y_pred1 = np.argmax(y_pred, axis=1)
y_pred1

array([9, 8, 8, ..., 8, 4, 6], dtype=int64)

In [43]:
y_val1 = np.argmax(y_val, axis=1)
y_val1

array([7, 8, 0, ..., 8, 2, 6], dtype=int64)

In [45]:
score = 0

for i, j in zip(y_pred1, y_val1):
    if i == j:
        score += 1
print(f'Accuracy: {score/len(y_val)}')

Accuracy: 0.6125


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)